# **🎙️ Proyecto de Reconocimiento de Voz a Texto usando Google Cloud Speech-to-Text**
Este proyecto utiliza la API de Google Cloud Speech-to-Text para transcribir audio en texto. A través de este proceso, se configuran las credenciales, se carga un archivo de audio y se realiza la transcripción.

🎯 Objetivo
El objetivo de este proyecto es implementar la tecnología de reconocimiento de voz a texto de Google Cloud para convertir archivos de audio en texto de manera precisa y eficiente. Utilizando esta API, se busca facilitar la transcripción de conversaciones o grabaciones en formato de texto, lo que tiene aplicaciones clave en campos como la accesibilidad, la automatización de transcripciones, y la creación de sistemas de interacción por voz más avanzados. Este proyecto permite explorar la integración de servicios de inteligencia artificial que ayudan a mejorar la interacción humana-máquina, proporcionando soluciones innovadoras en áreas como la atención al cliente, la toma de notas automáticas y los asistentes virtuales.

# Requisitos
Librerías necesarias:
google-cloud-speech: Para interactuar con la API de Speech-to-Text.
pydub: Para manejar y convertir archivos de audio a un formato compatible.
pydub.playback: Para reproducir archivos de audio.

In [19]:
pip install google-cloud-speech pydub pyaudio -q

Note: you may need to restart the kernel to use updated packages.


## Configuración de Autenticación

Configura tus credenciales para Google Cloud de forma similar a Text-to-Speech:

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"E:\proyecto\STT-GCP\CREDENCIAL.json"

# Inicialización del Cliente de Speech-to-Text
Creamos un cliente para interactuar con la API de Google Cloud Speech-to-Text:

In [3]:
from google.cloud import speech

# Crear el cliente
client = speech.SpeechClient()

## Conversión de Audio a Formato FLAC
Google Cloud Speech-to-Text funciona mejor con archivos en formato FLAC o LINEAR16. Usamos pydub para convertir el archivo de audio si es necesario:

In [18]:
from pydub import AudioSegment

def convertir_audio(input_path, output_path="output1.flac"):
    audio = AudioSegment.from_file(input_path)
    audio.export(output_path, format="flac")
    return output_path

## Grabación de Audio desde el Micrófono

Usamos la librería pyaudio para grabar audio desde el micrófono:

In [12]:
import pyaudio
import wave

# Configuración para la grabación
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
TIMEOUT = 2

# Inicialización de pyaudio
p = pyaudio.PyAudio()

def grabar_audio():
    stream = p.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True, frames_per_buffer=CHUNK)
    print("Grabando...")

    frames = []
    for i in range(0, int(RATE / CHUNK * TIMEOUT)):  # Graba durante TIMEOUT segundos
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()

    # Guardar el archivo de audio
    archivo_audio = "audio_grabado.wav"
    wf = wave.open(archivo_audio, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    return archivo_audio

## Transcripción del Audio con Google Cloud

Una vez que tenemos el archivo de audio grabado, podemos enviarlo a la API para transcribirlo a texto:

In [7]:
def transcribir_audio(audio_path):
    with open(audio_path, "rb") as audio_file:
        audio_content = audio_file.read()

    audio = speech.RecognitionAudio(content=audio_content)
    config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,  # Cambio aquí
    sample_rate_hertz=16000,  # Ajusta según la frecuencia de tu audio
    language_code="es-ES",   # Idioma español
)

    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        print(f"Transcripción: {result.alternatives[0].transcript}")


## Ejecución del Proyecto
- Grabar el audio: Usa la función grabar_audio() para capturar el sonido.
- Convertir a FLAC: Si es necesario, convertir_audio() convierte el archivo a FLAC.
- Transcribir el audio: transcribir_audio() envía el archivo a la API de Google para obtener la transcripción.

In [31]:
# Grabar audio y transcribir
archivo_audio = grabar_audio()
transcribir_audio(archivo_audio)

Grabando...
Transcripción: hola mundo
